Databricks notebook source
# este comando executa o notebook do caminho abaixo, dentro deste notebook. Ele pode estar na cloud, ou outro lugar, diretório e etc

In [ ]:
%run "Users/adolfoacarlos@gmail.com/secrets"

COMMAND ----------

In [ ]:
username = AZURE_SQL_USERNAME ## USER NAME DO DATABASE
password = AZURE_SQL_PASSWORD 

COMMAND ----------

# ler a tabela que está na Azure -
# utilizar a sparksession

In [ ]:
from pyspark.sql import SparkSession

COMMAND ----------

url da conexão lá na azure
codifike-server  = nome do servidor
spark-sql = nome do banco de dados
nos projetos em geral essas duas variáveis são utilizadas e compõe a url

In [ ]:
url = "jdbc:sqlserver://codifike-server.database.windows.net:1433;databaseName=spark-sql"

COMMAND ----------

opções de conexão 
é o nome do usuário, senha do usuário e  o driver que está sendo utilizado , está em formato de dicionário para facilitar
poderia exitir outros tipos de conexão, como single sign on, e etc

In [ ]:
connection_properties = {
    "user": username,
    "password": password,
    "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

COMMAND ----------

ler a tabela do banco de dados e armazenar os dados em um dataframe

In [ ]:
df = spark.read.jdbc (url,'sale.user',properties=connection_properties)

COMMAND ----------

In [ ]:
df.show()

COMMAND ----------

antes de fazer a leitura da tabela, é necessário primeiro criar uma tabela temporária aqui para "armazenar as informações que serão lidas"

In [ ]:
df.createOrReplaceTempView('users_table')

COMMAND ----------

MAGIC %sql
MAGIC -- fazer um join da tabela, lendo na nuvem da azure através do databricks
MAGIC
MAGIC SELECT * FROM users_table limit 10
MAGIC
MAGIC

COMMAND ----------

ler a tabela novamente

In [ ]:
df.spark.read.jdbc(url,"sale.sales",properties = connection_properties)

COMMAND ----------

In [ ]:
# cria novamente a tabela temporária
df.createOrReplaceTempView('sales_table')

COMMAND ----------

MAGIC %sql
MAGIC
MAGIC SELECT * FROM sales_table limit 20;

COMMAND ----------

MAGIC %sql
MAGIC --fazendo join
MAGIC
MAGIC SELECT u.first_name, u.last_name, s.sale_date, s.sale_amount
MAGIC FROM  users_table as u
MAGIC INNER JOIN sales_table s
MAGIC ON (u.user_id = s.user_id)

COMMAND ----------

pegar o resultado de um select , ou um querie e gravar novamente no banco de dados
usar o python , para conseguir gravar o resultado da query em uma variável

In [ ]:
query  = """
SELECT u.first_name, u.last_name, s.sale_date, s.sale_amount
FROM  users_table as u
INNER JOIN sales_table s
ON (u.user_id = s.user_id)
"""

COMMAND ----------

In [ ]:
# executa em um novo dataframe, o resultado da minha querye
result_df = spark.sql(query)

COMMAND ----------

escreve no banco
passar o nome da tabela onde eu quero gravar junto com o schema
o modo overwrite deve ser utilizado apenas se a tabela não existir, caso a tabela EXISTA, usar o modulo "append"

In [ ]:
result_df.write.jdbc(url, "sale.sales_report", mode= "overwrite", properties=connection_peoperties)

COMMAND ----------

código para usar as secrets da azure para databricks full

In [ ]:
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient

key vault é como um cofre de segurança digital, para que isso não precise ser gravados e guardados em lugares inseguros, como máquinas locais e tudo fique protegido por criptografia

In [ ]:
# Defina suas credenciais do Azure , são como senhas, chaves e tokens de acesso ( para segurança)
tenant_id = 'AZURE-ID-TENANT' # representa a "organização na azure"
client_id = 'AZURE-ID-CLIENTE' # id do cliente que está tentando acessar 
client_secret = 'AZURE-ID-CLIENTE' # senha do id do cliente que está tentando acessar
vault_url = "https://codifike-spark-chaves.vault.azure.net" # CAMINHO ONDE ESTÃO AS MINHAS SECRETS # endereço do cofre onde estão armazenadas minhas chaves de acesso

In [ ]:
# Autentique-se e crie um cliente
credential = ClientSecretCredential(tenant_id=tenant_id , client_id=client_id,client_secret=client_secret)
client = SecretClient(vault_url=vault_url , credential = credential)

Recupere os segredos

In [ ]:
db_username = client.get_secret('AZURE-SQL-USERNAME').value
db_password = client.get_secret('AZURE-SQL-PASSWORD').value